In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("LargeRowBenchmark").getOrCreate()


# UDF
def udf_upper(text):
    return text.upper()


spark.udf.register("udf_upper", udf_upper)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/10/18 12:52:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


<function __main__.udf_upper(text)>

In [2]:
df = spark.read.parquet(f"large_string_5row.parquet")
df.createOrReplaceTempView("T")
df.printSchema()
print("n_row:", df.count(), "n_col:", len(df.columns))
# df.show(1)

root
 |-- id: long (nullable = true)
 |-- 1m_string: string (nullable = true)
 |-- 10m_string: string (nullable = true)
 |-- 20m_string: string (nullable = true)
 |-- 30m_string: string (nullable = true)
 |-- 40m_string: string (nullable = true)
 |-- 50m_string: string (nullable = true)
 |-- 60m_string: string (nullable = true)
 |-- 70m_string: string (nullable = true)
 |-- 80m_string: string (nullable = true)
 |-- 90m_string: string (nullable = true)
 |-- 100m_string: string (nullable = true)

n_row: 5 n_col: 12


24/10/18 12:52:50 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


In [ ]:
# builtin: 50m_string on large_string_5row.parquet
# builtin: 40m_string on large_string_10row.parquet
spark.sql("SELECT upper(50m_string) FROM T").write.parquet("output_1.parquet", mode="overwrite")

In [ ]:
# UDF: 30m_string on large_string_5row.parquet
# UDF: 20m_string on large_string_10row.parquet
spark.sql("SELECT udf_upper(30m_string) FROM T").write.parquet("output_2.parquet", mode="overwrite")

# check

In [ ]:
# check
df_output = spark.read.parquet("output_2.parquet")
df_output.show(1)

In [5]:
spark.stop()